In [20]:
from StemLemPipe import phrases2lower, phrases_without_excess_symbols, phrases_transform, text2sentences, split_by_words, sentence_split, create_stemmer_lemmer, words_to_ngrams_list, sum_phrases, wordlist2set, stopwords, StemLemPipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
from nltk.corpus import stopwords
from collections import Counter
from tqdm import tqdm
import fasttext.util
import pandas as pd
import numpy as np
import matplotlib
import fasttext
import pickle
import string
import optuna
import time
import shap
import dill
import ast
import re

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import *
import spacy
import nltk

Загружаем и инициализируем необходимые данные, модели

In [2]:
num_folds = 5
random_state = 42
scoring = ['neg_mean_absolute_error', 'neg_mean_absolute_percentage_error']

In [6]:
result_df_all = pd.read_excel('results_4.xlsx', index_col=0)

In [4]:
fit_columns = ['count_skills', 'schedule_полный рабочий день', 'schedule_свободный график',
       'schedule_сменный график', 'schedule_удаленная работа',
       'schedule_частичная занятость', 'education_name_высшее (бакалавр)',
       'education_name_любое', 'education_name_неполное высшее',
       'education_name_среднее', 'education_name_среднее профессиональное',
       'city_id_10', 'city_id_16', 'city_id_174', 'city_id_2', 'city_id_268',
       'city_id_3', 'city_id_36', 'city_id_57', 'city_id_6',
       'city_id_Остальные', 'required_experience_Нет опыта',
       'required_experience_От 1 года до 3 лет',
       'required_experience_От 3 до 6 лет', 'vector_skills','vector_professions']

In [8]:
base_data = pd.read_csv('students_vacancies_10Nov2023_clear.csv', index_col=0)

In [9]:
base_data.work_skills = base_data.work_skills.apply(ast.literal_eval)
base_data.vector_skills = base_data.vector_skills.apply(ast.literal_eval)
base_data.vector_professions = base_data.vector_professions.apply(ast.literal_eval)

In [10]:
dim = 100

ft = fasttext.load_model('cc.ru.300.bin')

fasttext.util.reduce_model(ft, dim)

Лучшее значение MAPE на 5 фолдах 23.52

### Exp 1: clear, stemming, lemmatization

Попробуем преобработать данные перед получением эмбеддингов: сделаем лемматизацию, стемминг, а также очистим от стоп слов

In [17]:
def spacy_process(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

def nltk_process(text):
    #Tokenization
    nltk_tokenList = word_tokenize(text)
    
    #Stemming
    nltk_stemedList = []
    for word in nltk_tokenList:
        nltk_stemedList.append(s_stemmer.stem(word))
    
    #Lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in nltk_stemedList])

In [18]:
data = base_data.copy()

In [44]:
#Замена знаков пунктуации
data['custom_position'] = data.custom_position.apply(lambda value: re.sub(r'[^\w\s]', ' ', value))

#Удаление лишних пробелов
data['custom_position'] = data.custom_position.apply(lambda value: re.sub('\s+', ' ', value).strip())

In [45]:
words  = []

for sublist in [i.split(' ') for i in data.custom_position.to_list()]:
    for item in sublist:
        words.append(item)
        
cnt_df = pd.DataFrame.from_dict(Counter(words), orient='index').reset_index()
cnt_df.sort_values(by=0, ascending=False).head(20)

,index,0
26,по,11062
32,менеджер,7965
20,продавец,7872
40,на,5859
11,водитель,5712
9,в,4988
77,кассир,4978
47,оператор,4816
17,с,4160
36,продажам,3301


In [46]:
#Заменяем ка и ца
data['custom_position'] = data['custom_position'].replace({' ца ':' ', ' ка ':' '}, regex=True)

#Очищаем от стоп слов
stopwords = stopwords.words("russian")
data.custom_position = data.custom_position.apply(lambda value: ' '.join([word for word in value.split(' ') if word not in stopwords]))

In [47]:
nlp = spacy.load('ru_core_news_sm')

C:\Users\User\anaconda3\envs\myenv_py_3_8_dl\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [48]:
%%time

data['spacy_custom_position'] = data.custom_position.apply(lambda value: spacy_process(value))

CPU times: total: 3min 35s
Wall time: 3min 35s


In [49]:
s_stemmer = SnowballStemmer(language='russian')

In [50]:
%%time

data['nltk_custom_position'] = data.custom_position.apply(lambda value: nltk_process(value))

CPU times: total: 11.2 s
Wall time: 11.2 s


In [51]:
stem_lem = create_stemmer_lemmer(lemmatizer_backend='pymorphy', stemmer_backend='snowball')

In [52]:
%%time

data['stem_custom_position'] = phrases_transform(data.custom_position.to_list(), func = stem_lem)

CPU times: total: 30.9 s
Wall time: 30.9 s


In [53]:
data[data['nltk_custom_position']!=data['spacy_custom_position']].shape

(53186, 43)

In [54]:
data[data['nltk_custom_position']!=data['stem_custom_position']].shape

(6064, 43)

In [55]:
data[data['spacy_custom_position']!=data['stem_custom_position']].shape

(53074, 43)

In [56]:
data.to_csv('students_vacancies_10Nov2023_clear_ext_feat.csv')

In [23]:
data = pd.read_csv('students_vacancies_10Nov2023_clear_ext_feat.csv', index_col=0)

In [24]:
data.work_skills = data.work_skills.apply(ast.literal_eval)
data.vector_skills = data.vector_skills.apply(ast.literal_eval)
data.vector_professions = data.vector_professions.apply(ast.literal_eval)

In [57]:
model = CatBoostRegressor(verbose=0)

results = []
names = []
times = []

for custom_position_name in ['nltk_custom_position', 'spacy_custom_position', 'stem_custom_position']:
    
    vector_professions = []

    for words in data[custom_position_name].values.tolist():
        vector = np.mean([ft.get_word_vector(word) for word in words.split(' ')], axis=0).tolist()
        vector_professions.append(vector)
        
    y = data.salary_from
    X = data.drop('salary_from', axis=1)
    
    X['vector_professions'] = vector_professions
    
    X = X[fit_columns]
    
    vector_professions_df = pd.DataFrame(X.vector_professions.tolist(), columns=[f"feature_prof_{c}" for c in range(1, 100+1)])
    X.reset_index(drop=True, inplace=True)
    vector_professions_df.reset_index(drop=True, inplace=True)
    X = pd.concat([X, vector_professions_df], axis=1)

    vector_skills_df = pd.DataFrame(X.vector_skills.tolist(), columns=[f"feature_skill_{c}" for c in range(1, 100+1)])
    X.reset_index(drop=True, inplace=True)
    vector_skills_df.reset_index(drop=True, inplace=True)
    X = pd.concat([X, vector_skills_df], axis=1)
    
    X = X.drop(['vector_professions', 'vector_skills'], axis=1)
    
    start = time.time()
    kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

    scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)
    
    results.append(scores)
    names.append(custom_position_name+'_ctb')
    end = time.time()
    times.append(end - start)

In [58]:
result_df = pd.DataFrame(
    {'model_name': names,
     'MAE': [round(np.mean(np.abs(i['test_neg_mean_absolute_error'])),1) for i in results],
     'MAPE': [round(np.mean(np.abs(i['test_neg_mean_absolute_percentage_error']))*100,2) for i in results],
     'fit_time': [round(i,3) for i in times]
    })

result_df['stage'] = 'Exp 1'
result_df = result_df[['stage','model_name','MAE','MAPE','fit_time']]

In [59]:
result_df_all = pd.concat([result_df_all, result_df], axis=0)
result_df_all

,stage,model_name,MAE,MAPE,fit_time
0,baseline,LR,16283.3,29.53,1.720
1,baseline,RidgeRegression,16267.7,29.49,0.880
2,baseline,LassoRegression,16266.6,29.49,13.634
3,baseline,KNNRegression,14986.2,27.60,11.166
4,baseline,DecisionTreeRegressor,15059.2,28.39,142.998
5,baseline,CatBoostRegressor,12754.2,23.52,83.855
0,Exp 1,nltk_custom_position_ctb,12615.5,23.24,83.323
1,Exp 1,spacy_custom_position_ctb,12580.7,23.17,83.847
2,Exp 1,stem_custom_position_ctb,12616.2,23.25,83.539


In [60]:
result_df_all.to_excel('results_2.xlsx')

### Exp 2: подаем текстовые признаки в catboost

In [42]:
skills_cols = ['skill_1', 'skill_2', 'skill_3', 'skill_4', 'skill_5', 'skill_6','skill_7', 'skill_8', 'skill_9', 'skill_10']

ctb_fit_columns = ['count_skills', 'schedule_полный рабочий день', 'schedule_свободный график',
       'schedule_сменный график', 'schedule_удаленная работа',
       'schedule_частичная занятость', 'education_name_высшее (бакалавр)',
       'education_name_любое', 'education_name_неполное высшее',
       'education_name_среднее', 'education_name_среднее профессиональное',
       'city_id_10', 'city_id_16', 'city_id_174', 'city_id_2', 'city_id_268',
       'city_id_3', 'city_id_36', 'city_id_57', 'city_id_6',
       'city_id_Остальные', 'required_experience_Нет опыта',
       'required_experience_От 1 года до 3 лет',
       'required_experience_От 3 до 6 лет']

results = []
names = []
times = []

#### Все текстовые признаки в ctb

#### Все текстовые (исходные) признаки в ctb, компентенции одним признаком

In [43]:
y = data.salary_from
X = data.drop('salary_from', axis=1)

X = X[ctb_fit_columns + ['custom_position', 'work_skills_str']]

X['work_skills_str'] = X['work_skills_str'].fillna('нет')
X['work_skills_str'] = X['work_skills_str'].astype('str')

X['custom_position'] = X['custom_position'].astype('str')

model = CatBoostRegressor(verbose=0, text_features=['custom_position', 'work_skills_str'])

kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('ctb_with_text_feat_v1')
end = time.time()
times.append(end - start)

#### Все текстовые (леммы названий) признаки в ctb, компентенции одним признаком

In [47]:
y = data.salary_from
X = data.drop('salary_from', axis=1)

X = X[ctb_fit_columns + ['spacy_custom_position', 'work_skills_str']]

X = X.rename(columns={'spacy_custom_position':'custom_position'})

X['work_skills_str'] = X['work_skills_str'].fillna('нет')
X['work_skills_str'] = X['work_skills_str'].astype('str')

X['custom_position'] = X['custom_position'].astype('str')

model = CatBoostRegressor(verbose=0, text_features=['custom_position', 'work_skills_str'])

kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

start = time.time()

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('ctb_with_text_feat_v2')
end = time.time()
times.append(end - start)

#### Все текстовые (леммы названий) признаки в ctb, компентенции разными признаками

In [48]:
y = data.salary_from
X = data.drop('salary_from', axis=1)

X = X[ctb_fit_columns + ['spacy_custom_position'] + skills_cols]

X = X.rename(columns={'spacy_custom_position':'custom_position'})

X['custom_position'] = X['custom_position'].astype('str')

X[skills_cols] = X[skills_cols].fillna('нет')

model = CatBoostRegressor(verbose=0, text_features=['custom_position'] + skills_cols)

kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

start = time.time()

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('ctb_with_text_feat_v3')
end = time.time()
times.append(end - start)

#### Название в виде эмбеддингов FastText, а компетенции текстов в ctb (одним признаком)

In [68]:
y = data.salary_from
X = data.drop('salary_from', axis=1)

X = X[fit_columns + ['work_skills_str']]

X.vector_professions = X.vector_professions.apply(ast.literal_eval)

vector_professions_df = pd.DataFrame(X.vector_professions.tolist(), columns=[f"feature_prof_{c}" for c in range(1, 100+1)])

X.reset_index(drop=True, inplace=True)
vector_professions_df.reset_index(drop=True, inplace=True)
X = pd.concat([X, vector_professions_df], axis=1)

X = X.drop(['vector_professions', 'vector_skills'], axis=1)

X['work_skills_str'] = X['work_skills_str'].fillna('нет')
X['work_skills_str'] = X['work_skills_str'].astype('str')

model = CatBoostRegressor(verbose=0, text_features=['work_skills_str'])

kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

start = time.time()

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('ctb_with_text_feat_v4')
end = time.time()
times.append(end - start)

#### Название в виде эмбеддингов FastText, а компетенции текстов в ctb (разными признаками)

In [69]:
y = data.salary_from
X = data.drop('salary_from', axis=1)

X = X[fit_columns + skills_cols]

X.vector_professions = X.vector_professions.apply(ast.literal_eval)

vector_professions_df = pd.DataFrame(X.vector_professions.tolist(), columns=[f"feature_prof_{c}" for c in range(1, 100+1)])
X.reset_index(drop=True, inplace=True)
vector_professions_df.reset_index(drop=True, inplace=True)
X = pd.concat([X, vector_professions_df], axis=1)

X = X.drop(['vector_professions', 'vector_skills'], axis=1)

X[skills_cols] = X[skills_cols].fillna('нет')

model = CatBoostRegressor(verbose=0, text_features=skills_cols)

kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

start = time.time()

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('ctb_with_text_feat_v5')
end = time.time()
times.append(end - start)

In [70]:
result_df = pd.DataFrame(
    {'model_name': names,
     'MAE': [round(np.mean(np.abs(i['test_neg_mean_absolute_error'])),1) for i in results],
     'MAPE': [round(np.mean(np.abs(i['test_neg_mean_absolute_percentage_error']))*100,2) for i in results],
     'fit_time': [round(i,3) for i in times]
    })

result_df['stage'] = 'Exp 2'
result_df = result_df[['stage','model_name','MAE','MAPE','fit_time']]

In [71]:
result_df_all = pd.concat([result_df_all, result_df], axis=0)
result_df_all

,stage,model_name,MAE,MAPE,fit_time
0,baseline,LR,16283.3,29.53,1.720
1,baseline,RidgeRegression,16267.7,29.49,0.880
2,baseline,LassoRegression,16266.6,29.49,13.634
3,baseline,KNNRegression,14986.2,27.60,11.166
4,baseline,DecisionTreeRegressor,15059.2,28.39,142.998
5,baseline,CatBoostRegressor,12754.2,23.52,83.855
0,Exp 1,nltk_custom_position_ctb,12615.5,23.24,83.323
1,Exp 1,spacy_custom_position_ctb,12580.7,23.17,83.847
2,Exp 1,stem_custom_position_ctb,12616.2,23.25,83.539
0,Exp 3,multilingual_e5+ctb,13564.8,25.11,362.827


In [72]:
result_df_all.to_excel('results_3.xlsx')

### Exp 3: использование других эмбеддингов

In [10]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, CrossEncoder, LoggingHandler, losses, InputExample, util

In [11]:
results = []
names = []
times = []

#### SentenceTransformer

In [12]:
model_name = 'intfloat/multilingual-e5-large'

model = SentenceTransformer(model_name)

In [13]:
data['text_desc'] = data['custom_position'] + ' ' + data['work_skills_str']
data['text_desc'] = data['text_desc'].fillna('')

In [14]:
%%time

embedding_e5 = model.encode(data['text_desc'].values.tolist(), convert_to_tensor=True, device='cuda')

CPU times: total: 2min 12s
Wall time: 1min 58s


In [15]:
numpy_array = embedding_e5.cpu().detach().numpy()

df = pd.DataFrame(numpy_array)

In [16]:
data.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [21]:
y = data.salary_from
X = data.drop('salary_from', axis=1)

In [24]:
X = X[fit_columns].drop(['vector_skills', 'vector_professions'], axis=1)

In [26]:
X.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [28]:
X = pd.concat([X, df], axis=1)

In [30]:
model = CatBoostRegressor(verbose=0)

In [32]:
start = time.time()
kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('multilingual_e5+ctb')
end = time.time()
times.append(end - start)

In [33]:
result_df = pd.DataFrame(
    {'model_name': names,
     'MAE': [round(np.mean(np.abs(i['test_neg_mean_absolute_error'])),1) for i in results],
     'MAPE': [round(np.mean(np.abs(i['test_neg_mean_absolute_percentage_error']))*100,2) for i in results],
     'fit_time': [round(i,3) for i in times]
    })

result_df['stage'] = 'Exp 3'
result_df = result_df[['stage','model_name','MAE','MAPE','fit_time']]

In [38]:
result_df_all = pd.concat([result_df_all, result_df], axis=0)
result_df_all

,stage,model_name,MAE,MAPE,fit_time
0,baseline,LR,16283.3,29.53,1.720
1,baseline,RidgeRegression,16267.7,29.49,0.880
2,baseline,LassoRegression,16266.6,29.49,13.634
3,baseline,KNNRegression,14986.2,27.60,11.166
4,baseline,DecisionTreeRegressor,15059.2,28.39,142.998
5,baseline,CatBoostRegressor,12754.2,23.52,83.855
0,Exp 1,nltk_custom_position_ctb,12615.5,23.24,83.323
1,Exp 1,spacy_custom_position_ctb,12580.7,23.17,83.847
2,Exp 1,stem_custom_position_ctb,12616.2,23.25,83.539
0,Exp 3,multilingual_e5+ctb,13564.8,25.11,362.827


In [39]:
result_df_all.to_excel('results_4.xlsx')

### Hyp tuning

In [25]:
results = []
names = []
times = []

vector_professions = []

for words in data['spacy_custom_position'].values.tolist():
    vector = np.mean([ft.get_word_vector(word) for word in words.split(' ')], axis=0).tolist()
    vector_professions.append(vector)

y = data.salary_from
X = data.drop('salary_from', axis=1)

X['vector_professions'] = vector_professions

X = X[fit_columns]

vector_professions_df = pd.DataFrame(X.vector_professions.tolist(), columns=[f"feature_prof_{c}" for c in range(1, 100+1)])
X.reset_index(drop=True, inplace=True)
vector_professions_df.reset_index(drop=True, inplace=True)
X = pd.concat([X, vector_professions_df], axis=1)

vector_skills_df = pd.DataFrame(X.vector_skills.tolist(), columns=[f"feature_skill_{c}" for c in range(1, 100+1)])
X.reset_index(drop=True, inplace=True)
vector_skills_df.reset_index(drop=True, inplace=True)
X = pd.concat([X, vector_skills_df], axis=1)

X = X.drop(['vector_professions', 'vector_skills'], axis=1)

NameError: name 'ft' is not defined

In [15]:
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 5, 14),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
    }

    model = CatBoostRegressor(**params, verbose=0)

    kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

    scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

    return np.mean(np.abs(scores['test_neg_mean_absolute_percentage_error']))

In [17]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
best_params = study.best_params
print("Лучшие гиперпараметры:", best_params)

In [ ]:
best_params = {'learning_rate': 0.0952755462771626, 'depth': 10, 'subsample': 0.9148320072886789, 'colsample_bylevel': 0.9986120180597968, 'min_data_in_leaf': 34}

In [ ]:
model = CatBoostRegressor(**best_params, verbose=0) 
model.fit(X, y)

In [20]:
with open('ctb_reg_best.dill', 'wb') as f:
    dill.dump(model, f)

Добавим запись в таблицу с результатами

In [31]:
results = []
names = []
times = []

model = CatBoostRegressor(**best_params, verbose=0)

kfold = KFold(n_splits=num_folds, random_state=random_state, shuffle=True)

start = time.time()

scores = cross_validate(model, X, y, cv=kfold, scoring=scoring, return_train_score=False)

results.append(scores)
names.append('ctb')
end = time.time()
times.append(end - start)

In [33]:
result_df = pd.DataFrame(
    {'model_name': names,
     'MAE': [round(np.mean(np.abs(i['test_neg_mean_absolute_error'])),1) for i in results],
     'MAPE': [round(np.mean(np.abs(i['test_neg_mean_absolute_percentage_error']))*100,2) for i in results],
     'fit_time': [round(i,3) for i in times]
    })

result_df['stage'] = 'Hyp tuning'
result_df = result_df[['stage','model_name','MAE','MAPE','fit_time']]

In [50]:
result_df_all = pd.concat([result_df_all, result_df], axis=0)

In [40]:
result_df_all.to_excel('results_5.xlsx')

### SHAP

In [ ]:
with open('ctb_reg_best.dill', 'rb') as handle:
    model = pickle.load(handle)

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X)

In [ ]:
shap.plots.bar(shap_values, max_display=20)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=20)

In [ ]:
shap.plots.waterfall(shap_values[1])

In [ ]:
shap.plots.force(shap_values[1], matplotlib=matplotlib)